In [1]:
import gurobipy as gp
from gurobipy import GRB



def solve_SHIMRITS_adversary(
    bar_p,
    hat_p,
    edges,
    source,
    sink,
    Gamma,
    gurobi_output=False,
):
    n = len(bar_p)

    nodes = set()
    for i, j in edges:
        nodes.add(i)
        nodes.add(j)
    V = sorted(nodes)
    
    #Create Pi group
    Pi = {0.0}
    for i in range(n):
        if hat_p[i] > 0:
            Pi.add(float(hat_p[i]))
    Pi = sorted(Pi)
    
    #The flow variables
    out_arcs = {i: [] for i in V}
    in_arcs = {i: [] for i in V}
    for (i, j) in edges:
        out_arcs[i].append((i, j))
        in_arcs[j].append((i, j))

    model = gp.Model("adv_compact")
    if not gurobi_output:
        model.Params.OutputFlag = 0

    phi = model.addVars(edges, lb=0, ub=1, name="phi")
    alpha = model.addVars(range(n), vtype=GRB.BINARY, name="alpha")
    t = model.addVar(lb=-GRB.INFINITY, name="t")

    model.update()


    
    for i in V:
        if i == source:
            model.addConstr(gp.quicksum(phi[e] for e in out_arcs[i]) == 1)
        elif i == sink:
            model.addConstr(gp.quicksum(phi[e] for e in in_arcs[i]) == 1)
        else:
            model.addConstr(
                gp.quicksum(phi[e] for e in out_arcs[i]) -
                gp.quicksum(phi[e] for e in in_arcs[i])
                == 0
            )

    for i in range(n):
        model.addConstr(
            alpha[i] == gp.quicksum(phi[e] for e in out_arcs.get(i, [])),
            name=f"alpha_def_{i}",
        )

    for pi in Pi:
        expr = pi * Gamma
        for i in range(n):
            expr += max(hat_p[i] - pi, 0.0) * alpha[i]
        model.addConstr(t <= expr, name=f"pi_constraint_{pi}")

    obj = t + gp.quicksum(alpha[i] * bar_p[i] for i in range(n))
    model.setObjective(obj, GRB.MAXIMIZE)

    model.optimize()

    if model.status != GRB.OPTIMAL:
        return dict(
            best_value=None,
            best_pi=None,
            path_arcs=None,
            phi=None,
            alpha=None,
        )

    phi_sol = {e: phi[e].X for e in edges}
    alpha_sol = [alpha[i].X for i in range(n)]
    best_value = model.ObjVal

    path_arcs = []
    current = source
    visited = set()
    while current != sink and current not in visited:
        visited.add(current)
        nxt_arc = None
        for e in out_arcs[current]:
            if phi_sol.get(e, 0) > 0.5:
                nxt_arc = e
                break
        if nxt_arc is None:
            break
        path_arcs.append(nxt_arc)
        current = nxt_arc[1]

    # compute minimizer pi after solving
    t_star = t.X
    best_pi = None
    best_f_pi = float("inf")
    for pi in Pi:
        f_pi = pi * Gamma
        for i in range(n):
            f_pi += max(hat_p[i] - pi, 0.0) * alpha_sol[i]
        if f_pi < best_f_pi + 1e-6:
            best_f_pi = f_pi
            best_pi = pi

    return dict(
        best_value=best_value,
        best_pi=best_pi,
        path_arcs=path_arcs,
        phi=phi_sol,
        alpha=alpha_sol,
    )


In [2]:
def solve_adversary_bruni(
    bar_p,
    hat_p,
    edges,
    source,
    sink,
    Gamma,
    gurobi_output=False,
):
    """
    Bruni-style adversarial subproblem formulation (Section 4.2):

        max sum_(i,j) bar_p[i] * alpha_ij + hat_p[i] * w_ij

    s.t. alpha defines a path from source to sink,
         w_ij <= xi_i,
         w_ij <= alpha_ij,
         sum_i xi_i <= Gamma,
         alpha_ij in {0,1},
         0 <= xi_i <= 1, w_ij >= 0.

    Parameters
    ----------
    bar_p : list[float]
        Nominal durations bar_p[i] for each activity node i.
    hat_p : list[float]
        Deviations hat_p[i] for each activity node i.
    edges : list[tuple[int, int]]
        List of directed arcs (i, j) in the extended project network.
    source : int
        Index of the source node.
    sink : int
        Index of the sink node.
    Gamma : float or int
        Budget parameter.
    gurobi_output : bool, optional
        If True, Gurobi's output flag is enabled.

    Returns
    -------
    result : dict
        {
          "value": float,
          "path_arcs": list[(i, j)],
          "alpha": dict[(i,j)] -> float,
          "xi": dict[i] -> float,
          "w": dict[(i,j)] -> float
        }
    """
    n = len(bar_p)
    nodes, out_arcs, in_arcs = _build_graph(edges)

    model = gp.Model("adversary_bruni")
    model.Params.OutputFlag = 1 if gurobi_output else 0

    # Binary path variables alpha_ij
    alpha = {   
        (i, j): model.addVar(vtype=GRB.BINARY, name=f"alpha_{i}_{j}")
        for (i, j) in edges
    }

    # Continuous w_ij >= 0
    w = {
        (i, j): model.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"w_{i}_{j}")
        for (i, j) in edges
    }

    # Activity delay variables xi_i in [0, 1]
    xi = {
        i: model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name=f"xi_{i}")
        for i in nodes
    }

    model.update()

    # Flow constraints: single path from source to sink
    # 1) Source: sum of outgoing arcs is 1
    model.addConstr(
        gp.quicksum(alpha[(i, j)] for (i, j) in out_arcs[source]) == 1,
        name="source_flow",
    )

    # 2) Sink: sum of incoming arcs is 1
    model.addConstr(
        gp.quicksum(alpha[(i, j)] for (i, j) in in_arcs[sink]) == 1,
        name="sink_flow",
    )

    # 3) Intermediate nodes: flow conservation
    for i in nodes:
        if i in (source, sink):
            continue
        model.addConstr(
            gp.quicksum(alpha[(i, j)] for (i, j) in out_arcs[i])
            - gp.quicksum(alpha[(u, v)] for (u, v) in in_arcs[i])
            == 0,
            name=f"flow_balance_{i}",
        )

    # Linking constraints w_ij <= xi_i and w_ij <= alpha_ij
    for (i, j) in edges:
        model.addConstr(w[(i, j)] <= xi[i], name=f"w_le_xi_{i}_{j}")
        model.addConstr(w[(i, j)] <= alpha[(i, j)], name=f"w_le_alpha_{i}_{j}")

    # Budget constraint: sum_i xi_i <= Gamma
    model.addConstr(
        gp.quicksum(xi[i] for i in nodes) <= Gamma,
        name="budget_Gamma",
    )

    # Objective: sum_(i,j) bar_p[i] * alpha_ij + hat_p[i] * w_ij
    obj_expr = gp.LinExpr()
    for (i, j) in edges:
        if 0 <= i < n:
            obj_expr += bar_p[i] * alpha[(i, j)] + hat_p[i] * w[(i, j)]

    model.setObjective(obj_expr, GRB.MAXIMIZE)
    model.optimize()

    if model.status != GRB.OPTIMAL:
        raise RuntimeError(
            f"Gurobi did not find an optimal solution. Status: {model.status}"
        )

    value = model.ObjVal
    alpha_sol = {e: alpha[e].X for e in edges}
    xi_sol = {i: xi[i].X for i in nodes}
    w_sol = {e: w[e].X for e in edges}

    path_arcs = _extract_path_from_binary_flow(alpha_sol, out_arcs, source, sink)

    return {
        "value": value,
        "path_arcs": path_arcs,
        "alpha": alpha_sol,
        "xi": xi_sol,
        "w": w_sol,
    }


In [3]:
def _build_graph(edges):
    nodes = set()
    for i,j in edges:
        nodes.add(i); nodes.add(j)
    out_arcs = {i: [] for i in nodes}
    in_arcs  = {i: [] for i in nodes}
    for (i,j) in edges:
        out_arcs[i].append((i,j))
        in_arcs[j].append((i,j))
    return nodes, out_arcs, in_arcs

def _extract_path_from_binary_flow(alpha_sol, out_arcs, source, sink):
    path = []
    cur = source
    visited = set()
    while cur != sink and cur not in visited:
        visited.add(cur)
        nxt = None
        for e in out_arcs[cur]:
            if alpha_sol[e] > 0.5:
                nxt = e
                break
        if nxt is None:
            break
        path.append(nxt)
        cur = nxt[1]
    return path





In [4]:
import os
import time
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from gurobipy import GRB


# ============================================================
# 1) read psplib single-mode .sm
# ============================================================

def _ints(tokens):
    return [int(x) for x in tokens]

def _index_of_line(lines, substr):
    for i, ln in enumerate(lines):
        if substr in ln:
            return i
    raise ValueError(substr)

def _rhs(line):
    return line.split(":",1)[1].strip()

def parse_psplib_sm(path):
    path = Path(path)
    with path.open() as f:
        lines = [ln.rstrip() for ln in f]

    nj = int(_rhs(lines[_index_of_line(lines,"jobs (incl. supersource")]))
    nr = int(_rhs(lines[_index_of_line(lines,"- renewable")]).split()[0])

    prec_off = _index_of_line(lines,"PRECEDENCE RELATIONS:")+2
    attr_off = _index_of_line(lines,"REQUESTS/DURATIONS")+3
    cap_off  = _index_of_line(lines,"RESOURCEAVAILABILITIES")+2

    jobs = [f"j{i+1}" for i in range(nj)]
    resources=[f"r{k+1}" for k in range(nr)]

    succs={}
    for ix,j in enumerate(jobs):
        toks = lines[prec_off+ix].split()
        succs[j]=[f"j{int(s)}" for s in toks[3:]]

    durations={}
    demands={}
    for ix,j in enumerate(jobs):
        toks=lines[attr_off+ix].split()
        durations[j]=int(toks[2])
        rd=_ints(toks[3:])
        for kk,res in enumerate(resources):
            demands[(j,res)]=rd[kk]

    cap_tokens=lines[cap_off].split()
    capacities={resources[k]:int(cap_tokens[k]) for k in range(nr)}

    return dict(
        jobs=jobs,
        resources=resources,
        succs=succs,
        durations=durations,
        demands=demands,
        capacities=capacities
    )


# ============================================================
# 2) convert to robust RCPSP sets
# ============================================================

def psplib_to_rcpsp_sets(data):
    jobs=data["jobs"]
    resources=data["resources"]
    succs=data["succs"]
    durations=data["durations"]
    demands=data["demands"]
    caps=data["capacities"]

    job_index={j:i for i,j in enumerate(jobs)}
    V=list(job_index.values())

    E=[]
    for jf,scl in succs.items():
        i=job_index[jf]
        for jt in scl:
            E.append((i,job_index[jt]))

    K=resources[:]

    bar_p={job_index[j]:durations[j] for j in jobs}
    hat_p={i:bar_p[i]/2.0 for i in V}

    r={}
    for j in jobs:
        i=job_index[j]
        for res in resources:
            r[i,res]=demands[(j,res)]

    Nk={res:caps[res] for res in resources}

    return V,E,K,bar_p,hat_p,r,Nk




# ============================================================
# 4) batch benchmark over folder
# ============================================================

import os
import time
import matplotlib.pyplot as plt
import pandas as pd
from gurobipy import GRB


def _extract_best_value(res, solver_name=""):
    """
    Try to extract an objective value from the result object `res`,
    regardless of its exact structure.
    """
    # If it's a dict, try common keys
    if isinstance(res, dict):
        for key in ["best_value", "rho", "obj", "objective", "value"]:
            if key in res:
                return res[key]

    # If it's a Gurobi model
    try:
        if hasattr(res, "ObjVal"):
            return res.ObjVal
    except Exception:
        pass

    # If all failed, return NaN
    print(f"[WARN] Could not extract objective for solver='{solver_name}'.")
    return float("nan")


def benchmark(folder, gammas=(3, 5, 5.5, 7)):
    rows = []
    files = [f for f in os.listdir(folder) if f.endswith(".sm")]

    for fn in files:
        full = os.path.join(folder, fn)
        data = parse_psplib_sm(full)
        V, E, K, bar_p, hat_p, r, Nk = psplib_to_rcpsp_sets(data)

        n = len(V)
        edges = E
        source = min(V)
        sink = max(V)

        bar = [bar_p[i] for i in range(n)]
        hat = [hat_p[i] for i in range(n)]

        for Gamma in gammas:
            # SHIMRITS
            t0 = time.perf_counter()
            res1 = solve_SHIMRITS_adversary(
                bar_p=bar,
                hat_p=hat,
                edges=edges,
                source=source,
                sink=sink,
                Gamma=Gamma,
                gurobi_output=False,
            )
            t1 = time.perf_counter() - t0

            rows.append(
                dict(
                    dataset=os.path.basename(folder),
                    n=n,
                    file=fn,
                    Gamma=Gamma,
                    solver="SHIMRITS",
                    runtime=t1,
                    val=res1["best_value"],
                    path_arcs=res1.get("path_arcs"),
                )
            )

            # BRUNI
            t0 = time.perf_counter()
            res2 = solve_adversary_bruni(
                bar_p=bar,
                hat_p=hat,
                edges=edges,
                source=source,
                sink=sink,
                Gamma=Gamma,
                gurobi_output=False,
            )
            t2 = time.perf_counter() - t0

            rows.append(
                dict(
                    dataset=os.path.basename(folder),
                    n=n,
                    file=fn,
                    Gamma=Gamma,
                    solver="bruni",
                    runtime=t2,
                    val=res2["value"],
                    path_arcs=res2.get("path_arcs"),
                )
            )

    return pd.DataFrame(rows)





In [6]:
def write_results_txt(df, out_path, default_dataset=None):
    df = df.copy()

    # Add dataset if missing
    if "dataset" not in df.columns:
        df["dataset"] = default_dataset if default_dataset is not None else "NA"

    # Optional columns that might be missing
    if "n" not in df.columns:
        df["n"] = "NA"
    if "path_arcs" not in df.columns:
        df["path_arcs"] = None

    # Sort only by columns that exist
    sort_cols = [c for c in ["dataset", "file", "Gamma", "solver"] if c in df.columns]
    if sort_cols:
        df = df.sort_values(sort_cols).reset_index(drop=True)
    else:
        df = df.reset_index(drop=True)

    with open(out_path, "w", encoding="utf-8") as f:
        f.write("Adversary benchmark results\n")
        f.write("=" * 80 + "\n\n")

        for _, row in df.iterrows():
            dataset = row.get("dataset", "NA")
            file_ = row.get("file", "NA")
            n = row.get("n", "NA")
            Gamma = row.get("Gamma", "NA")
            solver = row.get("solver", "NA")
            runtime = row.get("runtime", float("nan"))
            val = row.get("val", float("nan"))

            f.write(
                f"dataset={dataset} | file={file_} | n={n} | "
                f"Gamma={Gamma} | solver={solver} | "
                f"runtime={runtime:.6f}s | val={val}\n"
            )

            path = row.get("path_arcs", None)
            if isinstance(path, list):
                f.write(f"optimal_path_length={len(path)}\n")
                f.write(f"optimal_path_arcs={path}\n")
            else:
                f.write("optimal_path_arcs=None\n")

            f.write("-" * 80 + "\n")

    print(f"Wrote results to: {out_path}")







'''if __name__ == "__main__":
    GAMMAS_BY_FOLDER = {
        "js30path": tuple(range(1, 16)),      # 1..15
        "js60path": tuple(range(2, 31, 2)),   # 2,4,...,30
        "js90path": tuple(range(3, 46, 3)),   # 3,6,...,45
    }

    folders = ["js30path", "js60path", "js90path"]

    for folder in folders:
        if not os.path.isdir(folder):
            print(f"[WARN] folder not found, skipping: {folder}")
            continue

        gammas = GAMMAS_BY_FOLDER.get(folder)
        if gammas is None:
            print(f"[WARN] no Gamma definition for folder: {folder}, skipping")
            continue

        print(f"Running benchmark for folder: {folder}")
        print(f"Gammas: {gammas}")

        df = benchmark(folder, gammas=gammas)
        print(df)

        # 1) write txt first (includes optimal chosen path)
        out_txt = f"results_{folder}.txt"
        write_results_txt(df, out_path=out_txt)

        # 2) plots
        plot_box_per_folder(df, folder)
        plot_bar_mean_per_folder(df, folder)'''




if __name__ == "__main__":

    GAMMAS_BY_FOLDER = {
        "js30path": tuple(range(1, 16)),      # 1..15
        "js60path": tuple(range(2, 31, 2)),   # 2,4,...,30
        "js90path": tuple(range(3, 46, 3)),   # 3,6,...,45
    }

    folders = ["js30path", "js60path", "js90path"]

    all_results = {}   # keep dfs in memory for later plotting if wanted

    for folder in folders:
        if not os.path.isdir(folder):
            print(f"[WARN] folder not found, skipping: {folder}")
            continue

        gammas = GAMMAS_BY_FOLDER.get(folder)
        if gammas is None:
            print(f"[WARN] no Gamma definition for folder: {folder}, skipping")
            continue

        print("=" * 80)
        print(f"Running benchmark for folder: {folder}")
        print(f"Gammas: {gammas}")

        df = benchmark(folder, gammas=gammas)
        all_results[folder] = df

        # ---- write TXT (includes optimal path arcs) ----
        out_txt = f"results_{folder}.txt"
        write_results_txt(df, out_path=out_txt, default_dataset=folder)

        print(f"[OK] Finished folder: {folder}")
        print("=" * 80)



Running benchmark for folder: js30path
Gammas: (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
Wrote results to: results_js30path.txt
[OK] Finished folder: js30path
Running benchmark for folder: js60path
Gammas: (2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30)
Wrote results to: results_js60path.txt
[OK] Finished folder: js60path
Running benchmark for folder: js90path
Gammas: (3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45)
Wrote results to: results_js90path.txt
[OK] Finished folder: js90path
